In [4]:
import random

import boto3
from botocore.exceptions import ClientError

# Initialize Cognito client
cognito_client = boto3.client("cognito-idp", region_name="us-east-1")

# User Pool ID
user_pool_id = "us-east-1_GyTh6JS9k"


def list_users():
    try:
        response = cognito_client.list_users(
            UserPoolId=user_pool_id,
            Limit=60,  # Adjust this value based on your needs
        )
        return response["Users"]
    except ClientError as e:
        print(f"An error occurred: {e}")
        return []


def get_user_attributes(username):
    try:
        response = cognito_client.admin_get_user(
            UserPoolId=user_pool_id, Username=username
        )
        return response["UserAttributes"]
    except ClientError as e:
        print(f"An error occurred: {e}")
        return []


# Example usage
users = list_users()
for user in users:
    print(f"User: {user['Username']}")
    attributes = get_user_attributes(user["Username"])
    for attr in attributes:
        print(f"  {attr['Name']}: {attr['Value']}")
    print("---")


User: 64f88438-4001-70d3-f925-e10194b95201
  email: testuser@example.com
  email_verified: true
  name: Test User
  custom:qs_remain: 6
  custom:zipnum: 713108
  sub: 64f88438-4001-70d3-f925-e10194b95201
---
User: f4b8e458-7021-70fa-3340-7ac297f1739f
  email: keongw@gmail.com
  email_verified: true
  name: Ginger G.
  custom:qs_remain: 1
  sub: f4b8e458-7021-70fa-3340-7ac297f1739f
---
User: 34382408-e091-7042-2ef1-35caddb92028
  email: abdelwahabtamir@gmail.com
  name: T A
  sub: 34382408-e091-7042-2ef1-35caddb92028
---
User: 146894d8-d061-7027-cc0c-e043f1accab8
  email: aaron.cooper@brinc.io
  email_verified: true
  name: Attributes
  custom:qs_remain: 6
  sub: 146894d8-d061-7027-cc0c-e043f1accab8
---
User: 9408d448-e081-70ef-0fe1-ad7518c5ca91
  email: olivialee1@gmail.com
  email_verified: true
  name: Olivia Lee
  sub: 9408d448-e081-70ef-0fe1-ad7518c5ca91
---
User: c4785478-80a1-70dd-0fe0-b812992da52e
  email: adrian.luk@dttd.io
  email_verified: true
  name: Adrian Luk
  custom:zip

In [2]:
import boto3
from botocore.exceptions import ClientError

# Initialize Cognito client
cognito_client = boto3.client("cognito-idp", region_name="us-east-1")

# User Pool ID
user_pool_id = "us-east-1_GyTh6JS9k"


def create_user(username, email, name, zipnum=None, qs_remain=None):
    try:
        user_attributes = [
            {"Name": "email", "Value": email},
            {"Name": "name", "Value": name},
            {"Name": "email_verified", "Value": "true"},
        ]
        if zipnum:
            user_attributes.append({"Name": "custom:zipnum", "Value": str(zipnum)})
        if qs_remain:
            user_attributes.append(
                {"Name": "custom:qs_remain", "Value": str(qs_remain)}
            )

        response = cognito_client.admin_create_user(
            UserPoolId=user_pool_id,
            Username=username,
            UserAttributes=user_attributes,
            MessageAction="SUPPRESS",
        )
        print(f"User {username} created successfully")
        return response["User"]
    except ClientError as e:
        print(f"An error occurred while creating user: {e}")
        return None


def read_user(username):
    try:
        response = cognito_client.admin_get_user(
            UserPoolId=user_pool_id, Username=username
        )
        return response
    except ClientError as e:
        print(f"An error occurred while reading user: {e}")
        return None


def update_user(username, attributes):
    try:
        response = cognito_client.admin_update_user_attributes(
            UserPoolId=user_pool_id, Username=username, UserAttributes=attributes
        )
        print(f"User {username} updated successfully")
        return response
    except ClientError as e:
        print(f"An error occurred while updating user: {e}")
        return None


def delete_user(username):
    try:
        response = cognito_client.admin_delete_user(
            UserPoolId=user_pool_id, Username=username
        )
        print(f"User {username} deleted successfully")
        return response
    except ClientError as e:
        print(f"An error occurred while deleting user: {e}")
        return None


# Example usage:
# create_user('newuser', 'newuser@example.com', 'New User', '12345')
# user_info = read_user('newuser')
# update_user('newuser', [{'Name': 'name', 'Value': 'Updated Name'}])
# delete_user('newuser')


In [ ]:
def update_qs_remain(name, new_value):
    try:
        # First, find the user by name
        users = cognito_client.list_users(
            UserPoolId=user_pool_id, Filter=f'name = "{name}"'
        )

        if not users["Users"]:
            print(f"No user found with name: {name}")
            return None

        username = users["Users"][0]["Username"]

        # Now update the user's attributes
        response = cognito_client.admin_update_user_attributes(
            UserPoolId=user_pool_id,
            Username=username,
            UserAttributes=[
                {"Name": "custom:qs_remain", "Value": str(new_value)},
            ],
        )
        print(f"Successfully updated custom:qs_remain for user {name} to {new_value}")
        return response
    except ClientError as e:
        print(f"An error occurred while updating custom:qs_remain for user {name}: {e}")
        return None


update_qs_remain("cktc", 101)

In [3]:
def decrease_qs_remain(name):
    try:
        # First, find the user by name
        users = cognito_client.list_users(
            UserPoolId=user_pool_id, Filter=f'name = "{name}"'
        )

        if not users["Users"]:
            print(f"No user found with name: {name}")
            return None

        username = users["Users"][0]["Username"]

        # Get current qs_remain value
        user_attributes = cognito_client.admin_get_user(
            UserPoolId=user_pool_id, Username=username
        )["UserAttributes"]

        current_qs_remain = next(
            (
                attr["Value"]
                for attr in user_attributes
                if attr["Name"] == "custom:qs_remain"
            ),
            None,
        )

        if current_qs_remain is None:
            print(f"User {name} does not have a custom:qs_remain attribute")
            return None

        # Decrease qs_remain by 1
        new_qs_remain = max(
            0, int(current_qs_remain) - 1
        )  # Ensure it doesn't go below 0

        # Update the user's attributes
        response = cognito_client.admin_update_user_attributes(
            UserPoolId=user_pool_id,
            Username=username,
            UserAttributes=[
                {"Name": "custom:qs_remain", "Value": str(new_qs_remain)},
            ],
        )
        print(
            f"Successfully decreased custom:qs_remain for user {name} from {current_qs_remain} to {new_qs_remain}"
        )
        return response
    except ClientError as e:
        print(
            f"An error occurred while decreasing custom:qs_remain for user {name}: {e}"
        )
        return None


# Example usage:
decrease_qs_remain("cktc")


Successfully decreased custom:qs_remain for user cktc from 99 to 98


{'ResponseMetadata': {'RequestId': '46953142-a080-4907-925b-f55d3126c41f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Nov 2024 07:45:20 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '46953142-a080-4907-925b-f55d3126c41f'},
  'RetryAttempts': 0}}

In [4]:
read_user("04c864d8-1091-7040-819e-48828fa43816")

{'Username': '04c864d8-1091-7040-819e-48828fa43816',
 'UserAttributes': [{'Name': 'email', 'Value': 'chris@dttd.io'},
  {'Name': 'email_verified', 'Value': 'true'},
  {'Name': 'name', 'Value': 'cktc'},
  {'Name': 'custom:qs_remain', 'Value': '98'},
  {'Name': 'custom:zipnum', 'Value': '1928'},
  {'Name': 'sub', 'Value': '04c864d8-1091-7040-819e-48828fa43816'}],
 'UserCreateDate': datetime.datetime(2024, 7, 30, 10, 44, 55, 963000, tzinfo=tzlocal()),
 'UserLastModifiedDate': datetime.datetime(2024, 11, 5, 15, 45, 20, 615000, tzinfo=tzlocal()),
 'Enabled': True,
 'UserStatus': 'CONFIRMED',
 'ResponseMetadata': {'RequestId': 'c3740af2-e88e-491e-b737-73cca20de48a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 05 Nov 2024 07:45:25 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '440',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c3740af2-e88e-491e-b737-73cca20de48a'},
  'RetryAttempts': 0}}

In [6]:
import random

import boto3


def create_user(email: str, name: str, qs_remain: int = 6):
    """
    Create a new user in Cognito user pool with specified attributes

    Args:
        email (str): User's email address
        name (str): User's name
        qs_remain (int): Initial number of remaining questions (default 100)

    Returns:
        dict: Response from Cognito or None if error occurs
    """
    try:
        # Create user with temporary password and random 6-digit zipnum
        temp_password = f"Welcome1!{random.randint(1000,9999)}"
        random_zipnum = str(random.randint(100000, 999999))

        response = cognito_client.admin_create_user(
            UserPoolId=user_pool_id,
            Username=email,
            TemporaryPassword=temp_password,
            UserAttributes=[
                {"Name": "email", "Value": email},
                {"Name": "name", "Value": name},
                {"Name": "custom:qs_remain", "Value": str(qs_remain)},
                {"Name": "custom:zipnum", "Value": random_zipnum},
                {"Name": "email_verified", "Value": "true"},
            ],
            MessageAction="SUPPRESS",
        )

        print(
            f"Successfully created user {name} with email {email} and zipnum {random_zipnum}"
        )
        print(f"Temporary password: {temp_password}")
        return response

    except ClientError as e:
        print(f"An error occurred while creating user {name}: {e}")
        return None


# Example usage:
create_user("testuser@example.com", "Test User")


An error occurred while creating user Test User: An error occurred (UsernameExistsException) when calling the AdminCreateUser operation: User account already exists


In [7]:
def check_group_exists(group_name: str) -> bool:
    """
    Check if a group exists in the Cognito user pool

    Args:
        group_name (str): Name of the group to check

    Returns:
        bool: True if group exists, False otherwise
    """
    try:
        response = cognito_client.get_group(
            GroupName=group_name, UserPoolId=user_pool_id
        )
        return True
    except cognito_client.exceptions.ResourceNotFoundException:
        return False
    except ClientError as e:
        print(f"An error occurred while checking group {group_name}: {e}")
        return False


# Example usage:
exists = check_group_exists("elements_advisors")
print(f"Group exists: {exists}")


Group exists: True


In [8]:
def create_group_if_not_exists(group_name: str, description: str = "") -> bool:
    """
    Create a group in the Cognito user pool if it doesn't already exist

    Args:
        group_name (str): Name of the group to create
        description (str): Optional description for the group

    Returns:
        bool: True if group was created or already exists, False if error occurred
    """
    try:
        # First check if group already exists
        if check_group_exists(group_name):
            print(f"Group {group_name} already exists")
            return True

        # Create the group if it doesn't exist
        response = cognito_client.create_group(
            GroupName=group_name, UserPoolId=user_pool_id, Description=description
        )
        print(f"Successfully created group {group_name}")
        return True

    except ClientError as e:
        print(f"An error occurred while creating group {group_name}: {e}")
        return False


# Example usage:
success = create_group_if_not_exists("elements_advisors", "Group for Elements advisors")
print(f"Group creation successful: {success}")


Group elements_advisors already exists
Group creation successful: True


In [9]:
def assign_user_to_group(username: str, group_name: str) -> bool:
    """
    Assign a user to a Cognito user pool group if both user and group exist

    Args:
        username (str): Username of the user to assign
        group_name (str): Name of the group to assign user to

    Returns:
        bool: True if assignment was successful, False if error occurred
    """
    try:
        # First verify group exists
        if not check_group_exists(group_name):
            print(f"Group {group_name} does not exist")
            return False

        # Verify user exists by trying to get user
        try:
            cognito_client.admin_get_user(UserPoolId=user_pool_id, Username=username)
        except cognito_client.exceptions.UserNotFoundException:
            print(f"User {username} does not exist")
            return False

        # Add user to group
        response = cognito_client.admin_add_user_to_group(
            UserPoolId=user_pool_id, Username=username, GroupName=group_name
        )
        print(f"Successfully added user {username} to group {group_name}")
        return True

    except ClientError as e:
        print(
            f"An error occurred while assigning user {username} to group {group_name}: {e}"
        )
        return False


# Example usage:
success = assign_user_to_group(
    "bertrand.chui@elements-advisors.com", "elements_advisors"
)
print(f"Group assignment successful: {success}")


Successfully added user bertrand.chui@elements-advisors.com to group elements_advisors
Group assignment successful: True


## User Creation and Group Assignment

In [14]:
import string
from pprint import pprint


def create_user_and_assign_group(name: str, email: str, group_name: str = None) -> dict:
    """
    Creates a new Cognito user with given name and email, and optionally assigns them to a group.

    Args:
        name (str): Full name of the user
        email (str): Email address of the user
        group_name (str, optional): Name of group to assign user to

    Returns:
        dict: Response containing user details and creation status
    """
    try:
        # Generate a temporary password
        temp_password = "".join(
            random.choices(
                string.ascii_letters + string.digits + string.punctuation, k=12
            )
        )

        # Generate random 6 digit zipnum
        random_zipnum = str(random.randint(100000, 999999))

        # Create user attributes
        user_attributes = [
            {"Name": "email", "Value": email},
            {"Name": "email_verified", "Value": "true"},
            {"Name": "name", "Value": name},
            {"Name": "custom:qs_remain", "Value": "6"},  # Default value
            {"Name": "custom:zipnum", "Value": random_zipnum},
        ]

        # Create the user
        response = cognito_client.admin_create_user(
            UserPoolId=user_pool_id,
            Username=email,
            UserAttributes=user_attributes,
            TemporaryPassword=temp_password,
            MessageAction="SUPPRESS",  # Suppress sending email
        )

        # If group name provided and exists, assign user to group
        if group_name and check_group_exists(group_name):
            group_assignment = assign_user_to_group(email, group_name)
            if not group_assignment:
                print(f"Warning: Failed to assign user to group {group_name}")

        return {
            "success": True,
            "user": response["User"],
            "temporary_password": temp_password,
            "group_assigned": group_name
            if group_name and check_group_exists(group_name)
            else None,
        }

    except ClientError as e:
        error_message = e.response["Error"]["Message"]
        print(f"Error creating user: {error_message}")
        return {"success": False, "error": error_message}


# Example usage:
result = create_user_and_assign_group(
    name="test",
    email="test@elements-advisors.com",
    group_name="element_advisors",
)
pprint(result)


{'group_assigned': None,
 'success': True,
 'temporary_password': '+0W|R{,umvP/',
 'user': {'Attributes': [{'Name': 'email',
                          'Value': 'test@elements-advisors.com'},
                         {'Name': 'email_verified', 'Value': 'true'},
                         {'Name': 'name', 'Value': 'test'},
                         {'Name': 'custom:qs_remain', 'Value': '6'},
                         {'Name': 'custom:zipnum', 'Value': '982126'},
                         {'Name': 'sub',
                          'Value': 'd40894a8-5091-708d-6011-d4f28a350b9e'}],
          'Enabled': True,
          'UserCreateDate': datetime.datetime(2024, 11, 21, 10, 39, 58, 989000, tzinfo=tzlocal()),
          'UserLastModifiedDate': datetime.datetime(2024, 11, 21, 10, 39, 58, 989000, tzinfo=tzlocal()),
          'UserStatus': 'FORCE_CHANGE_PASSWORD',
          'Username': 'd40894a8-5091-708d-6011-d4f28a350b9e'}}


In [15]:
# Print welcome message with login instructions
if result["success"]:
    print("\nWelcome to SnapInput!")
    print("----------------------------------------")
    print("Your account has been created successfully.")
    print("\nLogin Details:")
    print(f"Email: {result['user']['Attributes'][0]['Value']}")
    print(f"Temporary Password: {result['temporary_password']}")
    print("\nPlease visit https://d2.snapinput.com to log in")
    print("You will be prompted to change your password on first login")
    if result["group_assigned"]:
        print(f"\nYou have been assigned to group: {result['group_assigned']}")
    print("----------------------------------------")
else:
    print("\nError creating account:")
    print(result["error"])



Welcome to SnapInput!
----------------------------------------
Your account has been created successfully.

Login Details:
Email: test@elements-advisors.com
Temporary Password: +0W|R{,umvP/

Please visit https://d2.snapinput.com to log in
You will be prompted to change your password on first login
----------------------------------------
